In [1]:
import numpy as np
import subprocess
import os,glob,shutil
import tarfile
import sys


## Create SLHA files from a txname template

### Template file:

In [2]:
template = './templates/TDTM1F.template'

### Lines to be replaced

In [3]:
lines = ['   1000022     M1           # ~chi_10', '   1000024     M0               # ~chi_1+',
         'DECAY   1000024     W0               # chargino1']
newlines = ['   1000022     %1.3f           # ~chi_10','   1000024     %1.3f               # ~chi_1+',
         'DECAY   1000024     %1.3e               # chargino1']

#Check if lines exist
with open(template,'r') as f:
    flines = f.readlines()
    for l in lines:
        if not any(l in lf for lf in flines):
            print('Line:\n %s \n not found in %s' %(l,template))

### Parameters to generate files

In [ ]:
m1 = np.arange(100.,1100.,50.)
width = np.logspace(-17,-13,20)
# filename = 'TDTM1F_%1.0f_%1.1e_%1.0f_%1.0f_%1.1e_%1.0f.slha'
# tempFolder = 'TDTM1F'
# tarball = 'TDTM1F.tar.gz'
filename = 'TDTM2F_%1.0f_%1.1e_%1.0f_%1.0f.slha'
tempFolder = 'TDTM2F'
tarball = 'TDTM2F.tar.gz'

if not os.path.isdir(tempFolder):
    os.mkdir(tempFolder)

In [ ]:
with open(template,'r') as f:
    data = f.read()

files = []
for m in m1:    
    for w in width:
        
        #Define relations (constraints) between parameters
        m0 = m   #LSP mass
        
        #New file name
        # fnew = os.path.join(tempFolder,filename %(m,w,m0,m,w,m0))
        fnew = os.path.join(tempFolder,filename %(m,w,m0,m0))
        
        #Define values to be replaced
        values = [m0,m,w] #Values should be sorted according to the ordering in lines
        
        #Replace lines
        dataNew = data[:]
        for l,newl,v in zip(lines,newlines,values):          
            dataNew = dataNew.replace(l, newl %v)

        #Write new file
        with open(fnew,'w') as ff:
            ff.write(dataNew+'\n')
            fullPath = os.path.abspath(fnew)
            #Add ref cross-sections
            # c = subprocess.run("../smodels_utils/morexsecs/refxsecComputer.py -f %s -i 1000022" %fullPath,
                                # cwd="./",capture_output=True,shell=True)
            c = subprocess.run("../smodels_utils/morexsecs/refxsecComputer.py -f %s" %fullPath,
                                cwd="./",capture_output=True,shell=True)
            
            # print(c)

            #or compute xsecs:
            # c = subprocess.run("./smodelsTools.py xseccomputer -f %s -s 8 13 -8 -p" %fullPath,
                            #    cwd=os.path.expanduser('~/smodels'),capture_output=True,shell=True)
        # break
        files.append(fullPath)
    # break
print('%i files created' %len(files))

400 files created


### Create tarball and remove temp folder

In [6]:
with tarfile.open(tarball, "w:gz") as tar:
    tar.add(tempFolder, arcname='.')
shutil.rmtree(tempFolder)    